In [120]:
from docuverse.engines.data_template import DataTemplate
from docuverse.utils import read_config_file, get_orig_docid, get_param
from sentence_transformers import SentenceTransformer

In [44]:
# file1="../output/ibmsw-dev-short-milvus-dense-slate30m-512-100-10062024.cc.json"
# file2="../output/ibmsw-dev-short-milvus-dense-slate30m-512-100-10062024.dd.json"
# file1="../output/unified-search-short23k-milvus-dense-granite30m-512-100-20250709-4q.no-rerank.json"
# file2="../output/unified-search-short23k-chromadb-granite30m-512-100-20250709-4q.no-rerank.json"
# data_template="../config/ibm_search_beir_data.yml"
data_template="../../config/hindi_agro.yml"
file1="../output/hindi_agro_english-milvus-dense-granite149m-512-100-08262025.json"
file2="../output/hindi_agro_english-milvus-dense-bgem3-512-100-08262025.json"

data_config = read_config_file(data_template)

query_template = DataTemplate(data_config['query_format'])
doc_template = DataTemplate(data_config['data_format'])

In [87]:
from docuverse import SearchEngine

res1 = SearchEngine.read_output_(file1, query_template)
res2 = SearchEngine.read_output_(file2, query_template)

'id'

In [89]:
def get_correct_ranks(result):
    id_header = doc_template.text_header['id_header']
    keys = {id: 1 for id in get_param(result.question, query_template.relevant_header)}
    ranks = []
    for i, r in enumerate(result):
        rid = get_orig_docid(get_param(r, id_header))
        if rid in keys:
            ranks.append(i)
    if ranks==[]:
        return [100]
    return ranks

In [90]:
num_examples = len(res1)
top_k = 5
res = []
for i, (r1, r2) in enumerate(zip(res1, res2)):
    analysis = {'question': r1.question.as_dict(), 'res1_correct_ranks': get_correct_ranks(r1),
                'res2_correct_ranks': get_correct_ranks(r2)}
    res.append(analysis)

In [98]:
res1[0].question.as_dict()

{'text': "my farm has sandy loam soil and I'm thinking of planting long melon. Will it grow well here?",
 'id': '0',
 'relevant': ['1167942310109726123'],
 'answers': [''],
 'passages': ''}

In [99]:
def score(x):
    return x['res1_correct_ranks'][0] - x['res2_correct_ranks'][0]


aa=sorted(res, key=score, reverse=True)

In [ ]:
bb=[a for a in aa if score(a)>0]

In [121]:
m1=SentenceTransformer("ibm-granite/granite-embedding-english-r2")
m2=SentenceTransformer("BAAI/bge-m3")

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Flash Attention 2.0 only supports torch.float16 and torch.bfloat16 dtypes, but the current dype in ModernBertModel is torch.float32. You should run training or inference using Automatic Mixed-Precision via the `with torch.autocast(device_type='torch_device'):` decorator, or load the model with the `torch_dtype` argument. Example: `model = AutoModel.from_pretrained("openai/whisper-tiny", attn_implementation="flash_attention_2", torch_dtype=torch.float16)`


In [ ]:
a=m1.tokenizer("my cotton crop is facing some issues with pests. I've been using Copper oxychloride, but I'm not sure about the antidote if it causes any problems. Can you tell me what to do?")
b=m2.tokenizer("my cotton crop is facing some issues with pests. I've been using Copper oxychloride, but I'm not sure about the antidote if it causes any problems. Can you tell me what to do?")

In [126]:
m1.tokenizer.tokenize("my cotton crop is facing some issues with pests. I've been using Copper oxychloride, but I'm not sure about the antidote if it causes any problems. Can you tell me what to do?")

['my',
 'Ġcotton',
 'Ġcrop',
 'Ġis',
 'Ġfacing',
 'Ġsome',
 'Ġissues',
 'Ġwith',
 'Ġp',
 'ests',
 '.',
 'ĠI',
 "'ve",
 'Ġbeen',
 'Ġusing',
 'ĠCo',
 'pper',
 'Ġox',
 'ych',
 'lor',
 'ide',
 ',',
 'Ġbut',
 'ĠI',
 "'m",
 'Ġnot',
 'Ġsure',
 'Ġabout',
 'Ġthe',
 'Ġantid',
 'ote',
 'Ġif',
 'Ġit',
 'Ġcauses',
 'Ġany',
 'Ġproblems',
 '.',
 'ĠCan',
 'Ġyou',
 'Ġtell',
 'Ġme',
 'Ġwhat',
 'Ġto',
 'Ġdo',
 '?']

In [118]:
ii=114
print(res[ii])
print(res1[ii][0].as_dict())
print(res2[ii][4].as_dict())
print([i for i, d in enumerate(res1[ii]) if d['text'].find("Copper oxychloride")>=0])

{'question': {'text': "my cotton crop is facing some issues with pests. I've been using Copper oxychloride, but I'm not sure about the antidote if it causes any problems. Can you tell me what to do?", 'id': '114', 'relevant': ['298318831069241302'], 'answers': [''], 'passages': ''}, 'res1_correct_ranks': [40], 'res2_correct_ranks': [4]}
{'id': '7692093947940613276-0-115', 'text': 'Chapter: INSECT AND MITE PESTS Section: Management \nSpray 1875 ml chloropyriphos 20 EC in 500 litre water per acre.', 'title': '', 'score': 0.8793224692344666, 'rank': 0}
{'id': '298318831069241302-0-1520', 'text': 'The table outlines antidotes and treatments for various pesticides and herbicides. Here are the key points:\n\n1. **Copper oxychloride & Copper sulphate**: Treatment involves Dimercaprol (BAL) 3-4 mg/kg body weight, given deep intramuscular every 4 hours for 2 days and then twice for another 10 days. Sod. Bicarbonate 44-88 meq/ lit and D-penicillamine are also used.\n\n2. **Edifenphos (Hinosan)**